In [1]:
! pip3 install transformers datasets evaluate sentence-transformers


DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset, DatasetDict

try:
    dataset: DatasetDict = load_dataset("yelp_polarity")
    print("Dataset loaded successfully")
except Exception as e:
    print(f"An error occurred: {e}")


Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
from datasets import load_dataset_builder, get_dataset_split_names

print('split names:', get_dataset_split_names("yelp_polarity"))

ds_builder = load_dataset_builder("yelp_polarity")

print('dataset info')
print(ds_builder.info.description)
print(ds_builder.info.features)

print(dataset['train'])

In [ ]:
from datasets import concatenate_datasets
import random

def sample_from_dataset(_dataset, label, sample_size):
    filtered_dataset = _dataset.filter(lambda example: example['label'] == label)    
    random.seed(0)
    sampled_indices = random.sample(range(len(filtered_dataset)), sample_size)
    return filtered_dataset.select(sampled_indices)


def resize_dataset(_dataset):
    samples_label_0 = sample_from_dataset(_dataset, 0, 150)
    samples_label_1 = sample_from_dataset(_dataset, 1, 150)
    combined_samples = concatenate_datasets([samples_label_0, samples_label_1])
    combined_samples = combined_samples.shuffle(seed=42)
    return combined_samples

In [4]:
# insert your code here
train_dataset = resize_dataset(dataset['train'])
test_dataset = resize_dataset(dataset['test'])

In [5]:
train_dataset, test_dataset

(Dataset({
     features: ['text', 'label'],
     num_rows: 300
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 300
 }))

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

 Έγινε χρήση του bert-base-uncased, το οποίο είναι ένα version του BERT με 12 layers, 768 hidden units, και 12 attention heads. Μετατρέπει ολόκληρο το text σε μικρά γράμματα (γιαύτό "uncased").


In [6]:
# insert your code here
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the sequence classification model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization to each dataset
train_dataset = train_dataset.map(tokenize_function, batch_size=16)
test_dataset = test_dataset.map(tokenize_function, batch_size=16)
# ValueError: Expected input batch_size (8192) to match target batch_size (16).

In [8]:
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 300
})
Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 300
})


Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [9]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [164]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16)


# insert your code here
# optimizer

# scheduler
# from transformers import AdamW, get_scheduler
# 
# optimizer = AdamW(model.parameters(), lr=5e-5)  # You can adjust the learning rate
# 
# num_train_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
# scheduler = get_scheduler(
#     "cosine_with_restarts",
#     optimizer=optimizer,
#     num_warmup_steps=num_train_steps*0.1,  # Number of warmup steps; you can set this as a fraction of total training steps
#     num_training_steps=num_train_steps
# )

# etc

In [165]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [10]:
from torch.utils.data import DataLoader

# Assuming train_dataset has been tokenized and is ready for the DataLoader
def inspect_batch_sizes(dataset, batch_size=16):
    # Convert dataset to PyTorch format if necessary
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # Create DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size)

    # Get one batch from the DataLoader
    for batch in dataloader:
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['label']
        print("Input IDs batch size:", input_ids.shape)
        print("Attention mask batch size:", attention_mask.shape)
        print("Labels batch size:", labels.shape)
        break  # Only check the first batch for this example

# Call the function with the train_dataset
inspect_batch_sizes(train_dataset)


Input IDs batch size: torch.Size([16, 512])
Attention mask batch size: torch.Size([16, 512])
Labels batch size: torch.Size([16])


In [11]:
from torch.utils.data import DataLoader

# Assuming train_dataset has been tokenized and set up for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Get one batch from the DataLoader
batch = next(iter(dataloader))
input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['label']

# Assuming 'model' is already loaded and set to the correct mode (e.g., eval)
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Outputs are usually in the form of logits or probabilities
# Print the shape of outputs to check it
print("Output shape:", outputs.logits.shape)
print("Label shape:", labels.shape)


Output shape: torch.Size([16, 2])
Label shape: torch.Size([16])


# Δοκιμή με batch_size = 16, learning rate = 5e-5

In [140]:

trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.612310,0.713333
2,No log,0.313674,0.906667
3,No log,0.257079,0.920000


# Δοκιμή με batch_size = 8, learning rate = 5e-5

In [148]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8, learning_rate=5e-5)

# insert your code here
# optimizer

# scheduler
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)  # You can adjust the learning rate

num_train_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "cosine_with_restarts",
    optimizer=optimizer,
    num_warmup_steps=num_train_steps*0.1,  # Number of warmup steps; you can set this as a fraction of total training steps
    num_training_steps=num_train_steps
)

args.lr_scheduler = scheduler

# etc
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.262008,0.903333
2,No log,0.630501,0.873333
3,No log,0.508049,0.910000


TrainOutput(global_step=114, training_loss=0.11001695666396827, metrics={'train_runtime': 4345.7138, 'train_samples_per_second': 0.207, 'train_steps_per_second': 0.026, 'total_flos': 236799949824000.0, 'train_loss': 0.11001695666396827, 'epoch': 3.0})

# Δοκιμή με batch_size = 16, learning rate = 4e-5

In [169]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16, learning_rate=4e-5)

# insert your code here
# optimizer

# scheduler
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)  # You can adjust the learning rate

num_train_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "cosine_with_restarts",
    optimizer=optimizer,
    num_warmup_steps=num_train_steps*0.1,  # Number of warmup steps; you can set this as a fraction of total training steps
    num_training_steps=num_train_steps
)

# args.lr_scheduler = scheduler

# etc
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.538898,0.713333
2,No log,0.243724,0.923333
3,No log,0.222301,0.930000


TrainOutput(global_step=57, training_loss=0.39291666265119585, metrics={'train_runtime': 4507.9554, 'train_samples_per_second': 0.2, 'train_steps_per_second': 0.013, 'total_flos': 236799949824000.0, 'train_loss': 0.39291666265119585, 'epoch': 3.0})

# Δοκιμή με batch_size = 16, learning rate = 7e-5

In [12]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16, learning_rate=7e-5)

# insert your code here
# optimizer

# scheduler
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)  # You can adjust the learning rate

num_train_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "cosine_with_restarts",
    optimizer=optimizer,
    num_warmup_steps=num_train_steps*0.1,  # Number of warmup steps; you can set this as a fraction of total training steps
    num_training_steps=num_train_steps
)

# args.lr_scheduler = scheduler

# etc
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderC

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.409234,0.896667
2,No log,0.250530,0.900000
3,No log,0.261611,0.913333


TrainOutput(global_step=57, training_loss=0.3637952302631579, metrics={'train_runtime': 4438.4764, 'train_samples_per_second': 0.203, 'train_steps_per_second': 0.013, 'total_flos': 236799949824000.0, 'train_loss': 0.3637952302631579, 'epoch': 3.0})

β) Συνολικα σχόλια- παρατηρήσεις για τις υπερπαραμέτρους του μοντέλου:
Πώς το επηρεάζουν;
Ο καλύτερος συνδυασμός υπερπαραμέτρων είναι: batch_size = 16, learning rate = 4e-5.
Θα μπορούσαμε ομώς να συνάγουμε ως γενική παρατήρηση ότι αρχικά, μεγαλύτερο learning rate οδηγεί σε ταχύτερη σύγκλιση, ωστόσο και σε χειρότερη ακρίβεια. Βέβαια, μικρότερο learning rate απαιτεί περισσότερες εποχές. Απο την άλλη μεριά, μεγαλύτερο batch size παρατηρούμε για το συγκεκριμένο task πως έχει υψηλότερη ακρίβεια.


# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [1]:
# insert your code here (load dataset)
from datasets import load_dataset

piqa_dataset = load_dataset("piqa")

C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

Επιλεγμένα μοντέλα: "bert-base-uncased", "roberta-base", "distilbert-base-uncased", "albert-base-v2", "xlnet-base-cased"

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
from datasets import load_metric

class MultipleChoiceModel:
    def __init__(self, model_name, dataset, num_samples=100):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForMultipleChoice.from_pretrained(model_name)
        self.dataset = dataset['train'].select(range(num_samples))
        self.metric = load_metric("accuracy")

#Για την preprocess απο κάτω: Flattens the list of sentences and tokenizes them using the tokenizer.
#Returns tokenized examples grouped in pairs.
    def preprocess(self, examples):
        first_sentences = [[context] * 2 for context in examples["goal"]]
        question_headers = examples["sol1"], examples["sol2"]
        second_sentences = [
            [header for header in question_header] for question_header in zip(*question_headers)
        ]

        first_sentences = sum(first_sentences, [])
        second_sentences = sum(second_sentences, [])

        tokenized_examples = self.tokenizer(first_sentences, second_sentences, truncation=True, padding=True)

        return {
            k: [v[i:i + 2] for i in range(0, len(v), 2)]
            for k, v in tokenized_examples.items()
        }

    def compute_metrics(self, p):
        preds = p.predictions.argmax(-1)
        return self.metric.compute(predictions=preds, references=p.label_ids)
    
    def train(self):
        tokenized_datasets = self.dataset.map(self.preprocess, batched=True)

        training_args = TrainingArguments(
            output_dir="./results",
            evaluation_strategy="epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=3,
            weight_decay=0.01,
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_datasets,
            eval_dataset=tokenized_datasets,
            tokenizer=self.tokenizer,
            compute_metrics=self.compute_metrics,
        )

        trainer.train()
        return trainer.evaluate()

# Usage example with different models
models = ["bert-base-uncased", "roberta-base", "distilbert-base-uncased", "albert-base-v2", "xlnet-base-cased"]

for model_name in models:
    mc_model = MultipleChoiceModel(model_name, piqa_dataset)
    results = mc_model.train()
    print(f"Model: {model_name} - Results: {results}")


C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\kosti\AppData\Local\Temp\ipykernel_22052\3218469205.py:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.metric = load_metric("accuracy")
C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


...

Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.670776,0.630000
2,No log,0.664948,0.670000



KeyboardInterrupt



Καλύτερα αποτελέσματα λήφθηκαν από το μοντέλο ALBERT, για το οποίο γνωρίζουμε ότι ελαττώνει τον συνολικό αριθμό παραμέτρων σε σχέση με τα υπόλοιπα μοντέλα. Αυτό επιτυγχάνεται με το  embedding parameterization του μοντέλου που επιτρέπει ακόμα να διαχειριστεί πιο αποδοτικά μεγάλο όγκο από vocabulary, ενώ αντικαθιστά next sentence prediction (NSP) του BERT με το sentence order prediction (SOP), το οποίο προφανώς είναι αρκετά πιο εύκολο.

In [ ]:
# insert your code here (function for ending prediction)

## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [7]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

0.40488446

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [3]:
# insert your code here (load dataset)
from datasets import load_dataset

# Load the Truthful QA dataset
truthful_qa_dataset = load_dataset("truthful_qa", "generation")


Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [6]:
/# insert your code here
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np

# Φόρτωση του dataset
dataset = load_dataset('truthful_qa', 'generation')['validation']

# Επιλογή 100 δειγμάτων με τουλάχιστον 2 correct answers
filtered_dataset = [item for item in dataset if len(item['correct_answers']) >= 2][:100]

# Επιλογή sentence transformers models
models = [
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/all-distilroberta-v1',
    'sentence-transformers/all-MiniLM-L6-v2',
    'sentence-transformers/paraphrase-MiniLM-L12-v2',
    'sentence-transformers/quora-distilbert-multilingual',
    'sentence-transformers/stsb-roberta-base-v2'
]

# Συνάρτηση για τον υπολογισμό του accuracy
def calculate_accuracy(model_name, data):
    model = SentenceTransformer(model_name)
    correct_count = 0
    
    for item in data:
        question = item['question']
        best_answer = item['best_answer']
        correct_answers = item['correct_answers'][:2]  # Λαμβάνουμε τα πρώτα 2 correct answers
        incorrect_answer = item['incorrect_answers'][0]  # Λαμβάνουμε το πρώτο incorrect answer
        
        candidates = [best_answer] + correct_answers + [incorrect_answer]
        embeddings = model.encode(candidates)
        
        best_embedding = embeddings[0]
        correct_embeddings = embeddings[1:3]
        predicted_index = np.argmax([cosine_similarity([best_embedding], [emb])[0][0] for emb in correct_embeddings])
        
        if cosine_similarity([best_embedding], [correct_embeddings[predicted_index]])[0][0] > 0.95:
            correct_count += 1
    
    return correct_count / len(data)

# Υπολογισμός accuracy για κάθε μοντέλο
accuracies = {}
for model_name in models:
    accuracy = calculate_accuracy(model_name, filtered_dataset)
    accuracies[model_name] = accuracy

accuracies


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\kosti\anaconda3\envs\llm_env\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kosti\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'sentence-transformers/all-mpnet-base-v2': 0.83,
 'sentence-transformers/all-distilroberta-v1': 0.81,
 'sentence-transformers/all-MiniLM-L6-v2': 0.81,
 'sentence-transformers/paraphrase-MiniLM-L12-v2': 0.81,
 'sentence-transformers/quora-distilbert-multilingual': 0.91,
 'sentence-transformers/stsb-roberta-base-v2': 0.78}

Καλύτερη ακρίβεια: quora-distilbert-multilingual. Μοντέλο που έχει εκπαιδευτεί σε Quora data και είναι  βασισμένο στην αρχιτεκτονική DistilBERT.

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
# insert your code here (load dataset)
from datasets import load_dataset

dataset = load_dataset("winogrande")

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [ ]:
# insert your code here (load models)

In [ ]:
# insert your code here (create pipelines)

In [ ]:
# insert your code here (function for predicting best fill)